In [2]:
from elements.batteryModel import LithiumIonBattery

import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
from mat4py import loadmat
from scipy.linalg import block_diag, cholesky
from copy import copy

In [8]:
data = loadmat('./PANdata_P25.mat')
data['DYNData'].keys()

dict_keys(['script1', 'script2', 'script3', 'eta', 'Q'])

In [12]:
time    = np.array(data['DYNData']['script1']['time'] )  
deltat = time[1]-time[0]
time    = time-time[0] # start time at 0
current = np.array(data['DYNData']['script1']['current']) # discharge > 0; charge < 0.
voltage = np.array(data['DYNData']['script1']['voltage'])
soc     = np.array(data['DYNData']['script1']['soc'])

In [14]:
soc

array([1.        , 1.        , 1.        , ..., 0.09465054, 0.09465054,
       0.09465054])

In [19]:
%matplotlib qt

In [20]:
plt.plot(np.arange(10), np.sin(np.arange(10)))